# **Join the dots!**

## [Riddler Classic](https://fivethirtyeight.com/features/who-will-win-riddler-jeopardy/), July 2, 2021

### solution by [Laurent Lessard](https://laurentlessard.com)

In this challenge, the numbers from 1 to 11 are arranged in a circle in a particular order: 1, 4, 8, 7, 11, 2, 5, 9, 3, 6, 10. You then have to connect pairs of numbers with straight line segments that don’t intersect, and your score is the sum of the products of the joined numbers. For example, with the connections {1, 4}, {8, 10}, {3, 7}, {5, 9}, and {2, 11} (and the 6 left by itself), you get a score of 1·4 + 8·10 + 3·7 + 5·9 + 2·11, or 172.

The best score you can achieve with this ordering of 1 through 11 around the circle is 237, which you can get with the following connections: {6, 10}, {3, 4}, {7, 8}, {9, 11} and {2, 5} (and the 1 left by itself).

This got Friend-of-The-Riddler Tyler Barron and me thinking about possible extensions of this challenge. If you want the highest possible maximum score, then you can rearrange the numbers from 1 to 11 so that they are in numerical order around the circle. (With this arrangement, the maximum score is 250.)

But what if you want the lowest possible maximum score? That is, how can you order the numbers from 1 to 11 around the circle so that the maximum possible score is as low as possible? And what is the resulting score?

---

Let's enumerate all possible sets of $n$ pairs of brackets

In [96]:
import numpy as np
from itertools import permutations

In [97]:
cache = {}

def parentheses(n):
    if n == 0:
        return set([''])
    elif n in cache:
        return cache[n]
    else:
        par = set('(' + p + ')' for p in parentheses(n-1))
        for k in range(1,n):
            par.update(p+q for p in parentheses(k) for q in parentheses(n-k))
        cache[n] = par
        return par

In [111]:
n = 5
q = list(parentheses(n))

In [112]:
def arr2mat(arrangement):
    P1 = np.zeros((n,2*n))
    P2 = np.zeros((n,2*n))
    leftcount = 0
    ixlist = []
    for k,s in enumerate(arrangement):
        if s == '(':
            P1[leftcount,k] = 1
            ixlist.append(leftcount)
            leftcount = leftcount + 1
        elif s == ')':
            P2[ixlist.pop(),k] = 1
        else:
            print('error')
    return P1.T @ P2

Qlist = [arr2mat(arrangement) for arrangement in q]

In [113]:
Qbig = np.array([ np.reshape(Q,(-1,1),'F').T[0] for Q in Qlist ])

In [114]:
def f(x):
    xtmp = np.reshape(np.array(x),(-1,1))
    xbig = np.kron(xtmp,xtmp)
    return np.max((Qbig @ xbig).T[0])

def af(x):
    xtmp = np.reshape(np.array(x),(-1,1))
    xbig = np.kron(xtmp,xtmp)
    return np.argmax((Qbig @ xbig).T[0])

In [115]:
%%time
items = list(range(2,2*n+2))
results = [f(x) for x in permutations(items)]

Wall time: 1min 53s


In [116]:
ropt = np.argmin(results)
ropt

260218

In [117]:
ix = -1
for p in permutations(items):
    ix = ix+1
    if ix == ropt:
        popt = p
popt

(2, 9, 6, 8, 5, 7, 4, 11, 3, 10)

In [118]:
iopt = af(popt)
iopt

22

In [119]:
q[iopt]

'(())((()))'

In [121]:
copt = f(popt)
copt

185.0

In [125]:
f([8,5,7,4,11,10,1,2,9,6])

234.0